In [2]:
import h5py
import random
from os import path
from keras.models import Sequential, model_from_yaml, load_model
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler, ModelCheckpoint, TensorBoard
from keras.layers import *
from keras.backend.tensorflow_backend import set_session
import tensorflow as tf
import math
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [3]:
DATASET_DIR      = 'dataset'
dataset          = h5py.File(path.join(DATASET_DIR, 'dataset.hdf5'))

train_images     = dataset['image/train']
train_labels     = dataset['label/train']
train_centers    = dataset['center/train']

test_images      = dataset['image/test']
test_labels      = dataset['label/test']
test_centers     = dataset['center/test']


In [4]:
def generate_batches(images, labels, batch_size):
    while True:
        batch_indices = [(i, min(i + batch_size, len(labels))) for i in range(0, len(labels), batch_size)]
        random.shuffle(batch_indices)
        for start, end in batch_indices:
            image_batch, label_batch = images[start:end], labels[start:end]
            yield image_batch, label_batch

In [5]:
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
set_session(tf.Session(config=config))

In [6]:
benchmark_model = Sequential([\
                              Conv2D(\
                                     filters     = 8,\
                                     kernel_size = (5, 5),\
                                     strides     = (1, 1),\
                                     input_shape = (128, 128, 1)\
                                    ),\
                              MaxPooling2D(\
                                           pool_size = (3, 3)\
                                          ),\
                              LeakyReLU(\
                                        alpha = 0.05\
                                       ),\
                              Conv2D(\
                                     filters   = 8,\
                                     kernel_size = (5, 5),
                                     strides   = (1, 1)
                                    ),\
                              MaxPooling2D(\
                                           pool_size = (3, 3)\
                                          ),\
                              LeakyReLU(\
                                  alpha       = 0.05\
                              ),\
                              Conv2D(\
                                     filters   = 8,\
                                     kernel_size = (3, 3)\
                                    ),\
                              LeakyReLU(\
                                  alpha       = 0.05\
                              ),\
                              Flatten(),\
                              Dense(\
                                  units  = 1024,\
                                  activation  = 'relu'\
                              ),\
                              Dense(\
                                  units  = 1024,\
                                  activation  = 'relu'\
                              ),\
                              Dense(\
                                   units = 42
                                   )
                             ])

In [7]:
train_data = generate_batches(train_images, train_labels, 64)
test_data  = generate_batches(test_images, test_labels, 64)

In [8]:
benchmark_model.compile(
    optimizer = Adam(),
    loss      = 'mse'
)

In [11]:
benchmark_model.fit_generator(
    train_data,
    validation_data      = test_data,
    steps_per_epoch      = len(train_labels)/64,
    validation_steps     = len(test_labels)/64,
    verbose              = 1,
    epochs               = 100,
    callbacks            = [
        TensorBoard(log_dir='./benchmark'),
        ModelCheckpoint(
            filepath       = 'benchmark_model.hdf5',
            save_best_only = True
        )
    ])

Epoch 1/90
   3/3410 [..............................] - ETA: 1025s - loss: 47.5164

C:\Users\Wenjiec_Shen\AppData\Local\Continuum\Anaconda3\envs\tensorflow\lib\site-packages\keras\callbacks.py:120: UserWarning: Method on_batch_end() is slow compared to the batch update (0.484544). Check your callbacks.
  % delta_t_median)
C:\Users\Wenjiec_Shen\AppData\Local\Continuum\Anaconda3\envs\tensorflow\lib\site-packages\keras\callbacks.py:120: UserWarning: Method on_batch_end() is slow compared to the batch update (0.242756). Check your callbacks.
  % delta_t_median)


3251/3410 [===========================>..] - ETA: 16s - loss: 49.2923

C:\Users\Wenjiec_Shen\AppData\Local\Continuum\Anaconda3\envs\tensorflow\lib\site-packages\keras\callbacks.py:120: UserWarning: Method on_batch_end() is slow compared to the batch update (0.100842). Check your callbacks.
  % delta_t_median)


3254/3410 [===========================>..] - ETA: 16s - loss: 49.2737

C:\Users\Wenjiec_Shen\AppData\Local\Continuum\Anaconda3\envs\tensorflow\lib\site-packages\keras\callbacks.py:120: UserWarning: Method on_batch_end() is slow compared to the batch update (0.117622). Check your callbacks.
  % delta_t_median)
C:\Users\Wenjiec_Shen\AppData\Local\Continuum\Anaconda3\envs\tensorflow\lib\site-packages\keras\callbacks.py:120: UserWarning: Method on_batch_end() is slow compared to the batch update (0.112483). Check your callbacks.
  % delta_t_median)


3257/3410 [===========================>..] - ETA: 16s - loss: 49.2776

C:\Users\Wenjiec_Shen\AppData\Local\Continuum\Anaconda3\envs\tensorflow\lib\site-packages\keras\callbacks.py:120: UserWarning: Method on_batch_end() is slow compared to the batch update (0.105999). Check your callbacks.
  % delta_t_median)


3411/3410 [==============================] - 392s - loss: 49.0620 - val_loss: 52.4221
Epoch 2/90
1961/3410 [================>.............] - ETA: 139s - loss: 34.3394

KeyboardInterrupt: 

In [ ]:
def uvd_to_xyz(uvd):
    normalized_x = uvd[..., 0] / 640 - 0.5
    normalized_y = 0.5 - uvd[..., 1] / 480
    
    xyz = np.zeros(uvd.shape)
    xyz[..., 2] = uvd[..., 2]
    xyz[..., 0] = normalized_x * xyz[..., 2] * 1.08836710
    xyz[..., 1] = normalized_y * xyz[..., 2] * 0.817612648
    
    return xyz
def denormalize(label, center):
    label = label.copy()
    
    bounds = np.array([[0, 0], [1, 1]], dtype='float')
    bounds = bounds[None, ...].repeat(len(label), 0)
    bounds -= 0.5
    bounds *= 38
    bounds *= 525 / center[..., -1, None, None]
    bounds += center[..., None, -2::-1]
    bounds = bounds.astype(int)
    
    label[..., 0:2] /= (128, 128) / (bounds[..., None, 1, :] - bounds[..., None, 0, :])
    label[..., -1] *= 15.0
    label[..., -1] += center[..., -1, None]
    
    label[..., 0:2] += bounds[..., None, 0, :]
    
    label[..., -1] *= 10
    
    return label
def threshold(actual, predicted, threshold):
    max_distance = np.sqrt(np.sum((actual - predicted) ** 2, -1)).max(-1)
    count = np.sum((max_distance[..., None] < threshold), axis=0)
    return count / len(max_distance)

In [ ]:
default_model = load_model('default_model.hdf5')
plt.figure()
plt.xlabel('Distance threshold (mm)')
plt.ylabel('Test frames within distance (%)')
plt.xlim([0., 80.])
plt.ylim([0., 100.])
plt.yticks(np.arange(0, 100, 20))
plt.grid()
predictions = benchmark_model.predict(test_images, 64)
error = threshold(\
                  uvd_to_xyz(denormalize(test_labels[:].reshape(-1, 14, 3), test_centers[:])),\
                  uvd_to_xyz(denormalize(predictions.reshape(-1, 14, 3), test_centers[:])),\
                  np.linspace(0, 80, 1000)\
                 )
del predictions
plt.plot(np.linspace(0, 80, 1000), error * 100,label='Beseline')
predictions = default_model.predict(test_images, 64)
error = threshold(\
                  uvd_to_xyz(denormalize(test_labels[:].reshape(-1, 14, 3), test_centers[:])),\
                  uvd_to_xyz(denormalize(predictions.reshape(-1, 14, 3), test_centers[:])),\
                  np.linspace(0, 80, 1000)\
                 )
del predictions
plt.plot(np.linspace(0, 80, 1000), error * 100,label='Model')
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=3)